# TODO App with KIND - Interactive Guide

This notebook walks you through deploying and running the TODO application on a KIND (Kubernetes IN Docker) cluster.

## Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│  KIND Cluster (todo-app namespace)                              │
│                                                                 │
│   ┌──────────────┐      ┌──────────────┐      ┌──────────────┐  │
│   │    React     │      │   Go API     │      │   Export     │  │
│   │   (nginx)    │─────▶│   Service    │─────▶│   Service    │  │
│   │   web:80     │ /api │   api:8080   │ HTTP │ export:8081  │  │
│   └──────────────┘  /ws └──────────────┘      └──────────────┘  │
│                              │                                  │
│                              ▼                                  │
│                        ┌──────────────┐                         │
│                        │  PostgreSQL  │                         │
│                        │ postgres:5432│                         │
│                        └──────────────┘                         │
└─────────────────────────────────────────────────────────────────┘
```

## Ports

| Port | Service | Description |
|------|---------|-------------|
| 3000 | Web UI | React frontend |
| 8080 | API | Go REST API + WebSocket |
| 8081 | Export | Go export service (CSV/JSON) |

---
## 1. Check Current Status

First, let's see the current state of the cluster and app.

In [1]:
./status.sh

=== KIND Cluster Status ===
✖ Cluster 'kind' is not running
  Run: ./start-cluster.sh


---
## 2. Start the KIND Cluster

Create a KIND cluster configured for the DinD (Docker-in-Docker) environment.

This configures:
- API server accessible from the workspace container
- NodePort mappings for external access

In [2]:
./start-cluster.sh

Creating KinD cluster: kind
DinD sidecar: kind-app-example-10000-dind
Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.32.2) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
 ✓ Waiting ≤ 1m0s for control-plane = Ready ⏳7l
 • Ready after 17s 💚
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Not sure what to do next? 😅  Check out https://kind.sigs.k8s.io/docs/user/quick-start/

Cluster 'kind' is ready!
NodePorts 30080-30084, 80, 443 are accessible via: localhost

Kubernetes control plane is running at https://0.0.0.0:6443
CoreDNS is running at https://0.0.0.0:6443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


### Verify the cluster is running

In [3]:
./check-cluster.sh --expect=up

✔ SUCCESS: Cluster 'kind' is UP
NAME                 STATUS   ROLES           AGE   VERSION
kind-control-plane   Ready    control-plane   90s   v1.32.2


---
## 3. Build Docker Images

Build all the Docker images for the TODO app:
- `todo-web` - React frontend served by nginx
- `todo-api` - Go REST API with WebSocket support
- `todo-export` - Go export service

In [4]:
./build.sh

=== Building TODO App Docker Images ===

>>> Building API service...
DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  31.74kB
Step 1/11 : FROM golang:1.21-alpine AS builder
1.21-alpine: Pulling from library/golang
4f4fb700ef54: Pulling fs layer
41db7493d1c6: Pulling fs layer
54bf7053e2d9: Pulling fs layer
c6a83fedfae6: Pulling fs layer
4579008f8500: Pulling fs layer
b0059cac77ec: Download complete
4579008f8500: Download complete
b3f3effce7f5: Download complete
4f4fb700ef54: Download complete
41db7493d1c6: Download complete
c6a83fedfae6: Download complete
c6a83fedfae6: Pull complete
41db7493d1c6: Pull complete
54bf7053e2d9: Download complete
54bf7053e2d9: Pull complete
4579008f8500: Pull complete
4f4fb700ef54: Pull complete
Digest: sha256:2414035b086e3c42b99654c8b26e6f5b1b1598080d65fd03c7f4995

### Verify images are built

In [5]:
docker images | grep -E "todo-|REPOSITORY"

REPOSITORY     TAG           IMAGE ID       CREATED              SIZE
todo-web       latest        eaf3669f84f3   1 second ago         92.5MB
todo-export    latest        f0cf5151bf3b   42 seconds ago       24.5MB
todo-api       latest        7e8db8d4cd8c   About a minute ago   26.9MB


---
## 4. Deploy the App to KIND

Deploy all Kubernetes manifests:
- PostgreSQL (database with persistent volume)
- API service
- Export service
- Web frontend

In [7]:
./deploy-app.sh

=== Deploying TODO App to KIND ===

>>> Loading images into KIND cluster...
Image: "todo-api:latest" with ID "sha256:7e8db8d4cd8c594a8ae1fc2f50c511b67d573bcb37ad4a6da0e4e650944d88e4" not yet present on node "kind-control-plane", loading...
Image: "todo-export:latest" with ID "sha256:f0cf5151bf3ba060763850a3c7bc7c36443a964e11affa9ff152f08918906726" not yet present on node "kind-control-plane", loading...
Image: "todo-web:latest" with ID "sha256:eaf3669f84f3e2f1bbc69b016860acd18e9606a00dc1b526b25f020fd1c31645" not yet present on node "kind-control-plane", loading...

>>> Creating namespace 'todo-app'...
namespace/todo-app unchanged

>>> Applying Kubernetes manifests...
persistentvolumeclaim/postgres-pvc unchanged
configmap/postgres-config unchanged
configmap/postgres-init unchanged
deployment.apps/postgres unchanged
service/postgres unchanged

>>> Waiting for PostgreSQL to be ready...
deployment.apps/postgres condition met
deployment.apps/export-service unchanged
service/export-service u

In [10]:
kubectl port-forward svc/web 3000:80 -n todo-app &

[1] 4385


In [11]:
curl localhost:3000

Forwarding from 127.0.0.1:3000 -> 80
Forwarding from [::1]:3000 -> 80
Handling connection for 3000
Handling connection for 3000
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>TODO App</title>
    <script type="module" crossorigin src="/assets/index-7N9slrbU.js"></script>
    <link rel="stylesheet" crossorigin href="/assets/index-BVYZeIzG.css">
  </head>
  <body>
    <div id="root"></div>
  </body>
</html>


### Check pod status

Wait for all pods to be `Running`. This may take a minute.

In [14]:
kubectl get pods -n todo-app -w &

[2] 4443


### Check services

In [15]:
kubectl get svc -n todo-app

NAME                              READY   STATUS    RESTARTS   AGE
api-5c69c49fbc-kxvht              1/1     Running   0          5m11s
export-service-5f95c9fb8d-jvj9r   1/1     Running   0          5m12s
postgres-7d6576884-4gxqm          1/1     Running   0          5m32s
web-c47849589-8tpsr               1/1     Running   0          5m10s
NAME             TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
api              ClusterIP   10.96.66.103   <none>        8080/TCP   5m17s
export-service   ClusterIP   10.96.31.6     <none>        8081/TCP   5m18s
postgres         ClusterIP   10.96.32.2     <none>        5432/TCP   5m38s
web              ClusterIP   10.96.2.222    <none>        80/TCP     5m16s


---
## 5. Access the App

Start port-forwarding to access the app from your host browser.

**Important:** Uses `--address 0.0.0.0` so the app is accessible through the DinD sidecar's port mapping to your host.

In [16]:
./access-app.sh

Starting port-forward for TODO app...
  Web UI:  http://localhost:3000
  API:     http://localhost:8080
  Export:  http://localhost:8081

Error from server (NotFound): services "export" not found
Forwarding from 0.0.0.0:3000 -> 80
Forwarding from 0.0.0.0:8080 -> 8080

Port-forwards started in background.
Run './access-app-stop.sh' to stop them.
[1]-  Terminated              kubectl port-forward svc/web 3000:80 -n todo-app


### Now open in your browser:

- **Web UI:** [http://localhost:3000](http://localhost:3000)
- **API:** [http://localhost:8080/api/tasks](http://localhost:8080/api/tasks)
- **Export (JSON):** [http://localhost:8081/export?format=json](http://localhost:8081/export?format=json)
- **Export (CSV):** [http://localhost:8081/export?format=csv](http://localhost:8081/export?format=csv)

### Test the API directly

In [ ]:
curl -s http://localhost:8080/api/tasks | jq .

### Create a new task

In [ ]:
curl -s -X POST http://localhost:8080/api/tasks \
  -H "Content-Type: application/json" \
  -d '{"title": "Learn Kubernetes", "description": "Deploy apps to KIND"}' | jq .

### Export tasks as JSON

In [ ]:
curl -s http://localhost:8081/export?format=json | jq .

### Export tasks as CSV

In [ ]:
curl -s http://localhost:8081/export?format=csv

---
## 6. View Logs

Check logs from the different services.

In [ ]:
kubectl logs -n todo-app -l app=api --tail=20

In [ ]:
kubectl logs -n todo-app -l app=web --tail=20

In [ ]:
kubectl logs -n todo-app -l app=export --tail=20

---
## 7. Stop Port-Forwards

Stop the port-forwarding processes when done testing.

In [ ]:
./access-app-stop.sh

---
## 8. Cleanup

### Remove the app (keep the cluster)

In [ ]:
./remove-app.sh

### Delete the KIND cluster entirely

In [ ]:
./stop-cluster.sh

### Verify cluster is stopped

In [ ]:
./check-cluster.sh --expect=down

---
## Quick Reference

| Script | Description |
|--------|-------------|
| `./status.sh` | Show cluster, pods, services, port-forwards |
| `./start-cluster.sh` | Create KIND cluster |
| `./stop-cluster.sh` | Delete KIND cluster |
| `./check-cluster.sh` | Check if cluster is running |
| `./build.sh` | Build Docker images |
| `./deploy-app.sh` | Deploy TODO app to cluster |
| `./remove-app.sh` | Remove TODO app from cluster |
| `./access-app.sh` | Start port-forwards |
| `./access-app-stop.sh` | Stop port-forwards |